In [19]:
# Load the file that we will be using
with open("dexter_transcript/season_1/season_1_transcript_filtered.txt", 'r') as f:
    file_content = f.read()

# Print the number of characters in the file
print(f"Characters in File : {len(file_content)}")

Characters in File : 4103650


In [20]:
# Optionally print the beginning of the file
print(f"Beginning of File: \n{file_content[:500]}")

Beginning of File: 
Tonight's the night.

And it's going to happen again and again-- has to happen.

Nice night.

Miami is a great town. I love the Cuban food.

Pork sandwiches-- my favorite.

But I'm hungry for something different now.

There he is-- Mike Donovan.

He's the one.

You're mine now, so do exactly as I say.

What do you want?

I want you to be quiet.

Now drive.

Turn here.

You have to listen...

Do what I say.

Look.

No.

Uh, yes.

No, no!

It's horrible, isn't it? Isn't it?

Please...

Open your e


In [21]:
# Create a tokenizer dictionary with the maximum token length
# This helps to reduce the number of tokens needed for any arbitrary string
str_to_int = {}
token_count = 0
max_char_seq = 1 # Set the largest substring allowed to be tokenized
for length in range(1, max_char_seq + 1):
    sub_strings = []
    for i in range(len(file_content) - length + 1):
        sub_strings.append(file_content[i:i+length])
    for i, substring in enumerate(list(set(sub_strings)), token_count):
        str_to_int[substring] = i
    token_count = len(str_to_int)
    print(f"Length : {length} Token Count : {token_count}")

int_to_str = dict(zip(str_to_int.values(), str_to_int.keys()))
    
# Define the char -> int encoding function
def encode(s):
    encoding = []
    max_token_length = max_char_seq if len(s) >= max_char_seq else len(s)
    idx = 0
    while idx < len(s):
        if s[idx:max_token_length+idx] in str_to_int:
            encoding.append(str_to_int[s[idx:max_token_length + idx]])
            idx += max_token_length
        else:
            max_token_length -= 1
    return encoding

# Define the int -> char decoding function
def decode(int_list):
    return ''.join([int_to_str[i] for i in int_list])

# Simple test of the functions
print(encode("hello world!"))
print(decode(encode("hello world!")))

Length : 1 Token Count : 92
[23, 6, 53, 53, 46, 21, 19, 46, 18, 53, 30, 75]
hello world!
